**Objetivo:** Leer y estandarizar los datasets (SIES, DEMRE, IVM, georreferenciación, etc.)

- Cargar datos de todas las fuentes
- Verificar formatos y columnas clave
- Establecer claves de unión (`MRUN`, `RBD`, códigos de institución)
- Guardar versiones limpias en `data/processed/` (opcional)

In [2]:
import pandas as pd
import geopandas as gpd
from pathlib import Path
import os
import re
import math

# Configuraciones generales
pd.set_option("display.max_columns", None)

In [3]:
# Diccionario de regiones unificado
region_dict = {
    'REGION DE ARICA Y PARINACOTA': 'Arica y Parinacota',
    'REGION DE TARAPACA': 'Tarapacá',
    'REGION DE ANTOFAGASTA': 'Antofagasta',
    'REGION DE ATACAMA': 'Atacama',
    'REGION DE COQUIMBO': 'Coquimbo',
    'REGION DE VALPARAISO': 'Valparaíso',
    "REGION DEL LIBERTADOR GENERAL BERNARDO O'HIGGINS": "Lib. Gral B. O'Higgins",
    'REGION DEL MAULE': 'Maule',
    'REGION DE ÑUBLE': 'Ñuble',
    'REGION DEL BIOBIO': 'Biobío',
    'REGION DE LA ARAUCANIA': 'La Araucanía',
    'REGION DE LOS RIOS': 'Los Ríos',
    'REGION DE LOS LAGOS': 'Los Lagos',
    'REGION AISEN DEL GENERAL CARLOS IBAÑEZ DEL CAMPO': 'Aysén',
    'REGION DE MAGALLANES Y DE LA ANTARTICA CHILENA': 'Magallanes',
    'REGION METROPOLITANA DE SANTIAGO': 'Metropolitana',
    'REGIÓN DE ARICA Y PARINACOTA': 'Arica y Parinacota',
    'REGIÓN DE TARAPACÁ': 'Tarapacá',
    'REGIÓN DE ANTOFAGASTA': 'Antofagasta',
    'REGIÓN DE ATACAMA': 'Atacama',
    'REGIÓN DE COQUIMBO': 'Coquimbo',
    'REGIÓN DE VALPARAÍSO': 'Valparaíso',
    'REGIÓN DEL LIBERTADOR GRAL. BERNARDO O\'HIGGINS': "Lib. Gral B. O'Higgins",
    'REGIÓN DEL MAULE': 'Maule',
    'REGIÓN DE ÑUBLE': 'Ñuble',
    'REGIÓN DEL BIOBÍO': 'Biobío',
    'REGIÓN DE LA ARAUCANÍA': 'La Araucanía',
    'REGIÓN DE LOS RÍOS': 'Los Ríos',
    'REGIÓN DE LOS LAGOS': 'Los Lagos',
    'REGIÓN DE AYSÉN DEL GRAL. CARLOS IBÁÑEZ DEL CAMPO': 'Aysén',
    'REGIÓN DE MAGALLANES Y DE LA ANTÁRTICA CHILENA': 'Magallanes',
    'REGIÓN METROPOLITANA DE SANTIAGO': 'Metropolitana'
}

# Orden de regiones para visualización
orden_regiones = [
    'Arica y Parinacota', 'Tarapacá', 'Antofagasta', 'Atacama', 'Coquimbo',
    'Valparaíso', 'Metropolitana', "Lib. Gral B. O'Higgins", 'Maule', 'Ñuble',
    'Biobío', 'La Araucanía', 'Los Ríos', 'Los Lagos', 'Aysén', 'Magallanes'
]

# Diccionario para nombres de TIPO_DEPEN
tipodepen_dict = {
    1: 'Municipal',
    2: 'Particular Subvencionado',
    3: 'Particular Pagado',
    4: 'Corp. Administración Delegada',
    5: 'Servicio Local de Educación'
}

# Mapear códigos de región para set_C
regiones_dict_inv = {
    15: 'Arica y Parinacota',
    1: 'Tarapacá',
    2: 'Antofagasta',
    3: 'Atacama',
    4: 'Coquimbo',
    5: 'Valparaíso',
    6: 'Metropolitana',
    7: "Lib. Gral B. O'Higgins",
    8: 'Maule',
    16: 'Ñuble',
    9: 'Biobío',
    10: 'La Araucanía',
    11: 'Los Ríos',
    12: 'Los Lagos',
    13: 'Aysén',
    14: 'Magallanes'
}

### Lectura de Datos por Separado


In [4]:
# Lectura conjunto A
list( Path("../data/raw/2021").glob('*'))

[WindowsPath('../data/raw/2021/20230802_Matrícula_Ed_Superior_2021_PUBL_MRUN.csv'),
 WindowsPath('../data/raw/2021/A_INSCRITOS_PUNTAJES_PDT_2021_PUB_MRUN.csv'),
 WindowsPath('../data/raw/2021/establecimientos'),
 WindowsPath('../data/raw/2021/inmuebles_ies'),
 WindowsPath('../data/raw/2021/IVM_Establecimientos_2021.xlsx')]

In [5]:
# Definir el nombre del archivo
year = 2021
path_sies = Path(f"../data/raw/{year}")
path_set_A = path_sies / '20230802_Matrícula_Ed_Superior_2021_PUBL_MRUN.csv'
year_A = int(re.search(r'_(\d{4})_', str(path_set_A)).group(1))

# Definir las columnas clave
columnas_clave = [
    "mrun", "region_sede", "provincia_sede", "comuna_sede","cod_inst",
    "nomb_inst", "nomb_carrera", "tipo_inst_1","tipo_inst_2","tipo_inst_3", "nivel_global",
    "nivel_carrera_1", "anio_ing_carr_act", "anio_ing_carr_ori" , "forma_ingreso", "rango_edad"
]

# Leer el archivo con las columnas seleccionadas
set_a = pd.read_csv(path_set_A, sep=';', encoding='utf-8',usecols=columnas_clave) #añadir , usecols=columnas_clave para filtrar
set_a['NOMBRE_REGION_INGRESO'] = set_a['region_sede'].replace(region_dict)

# Mostrar las primeras filas
set_a.head(1)

,mrun,rango_edad,anio_ing_carr_ori,anio_ing_carr_act,tipo_inst_1,tipo_inst_2,tipo_inst_3,cod_inst,nomb_inst,nomb_carrera,region_sede,provincia_sede,comuna_sede,nivel_global,nivel_carrera_1,forma_ingreso,NOMBRE_REGION_INGRESO
0,19.0,20 a 24 años,2019,2019,Centros de Formación Técnica,Centros de Formación Técnica,Centros de Formación Técnica,536,CFT DE TARAPACA,TECNICO DE NIVEL SUPERIOR EN ENFERMERIA,Arica y Parinacota,ARICA,ARICA,Pregrado,Técnico de Nivel Superior,1- Ingreso Directo (regular),Arica y Parinacota


In [6]:
# Filtros Conjunto A

# Definir función para aplicar filtros
def aplicar_filtro(df, condiciones):
    """Filtra un DataFrame según un diccionario de condiciones"""
    for columna, valor in condiciones.items():
        df = df[df[columna] == valor]
    return df

# Diccionario de filtros
filtros_a = {
    "A0": {
        "anio_ing_carr_ori": year_A,
        "nivel_global": "Pregrado",
        "forma_ingreso": "1- Ingreso Directo (regular)"
    },
    "A": {
        "anio_ing_carr_ori": year_A,
        "nivel_global": "Pregrado",
        "forma_ingreso": "1- Ingreso Directo (regular)",
        "rango_edad": "15 a 19 años"
    },
    "A1": {
        "anio_ing_carr_ori": year_A,
        "nivel_global": "Pregrado",
        "forma_ingreso": "1- Ingreso Directo (regular)",
        "rango_edad": "15 a 19 años",
        "tipo_inst_1": "Universidades"
    }
}

# Aplicar todos los filtros y guardar los resultados
resultados_filtrados = {
    nombre: aplicar_filtro(set_a, condiciones)
    for nombre, condiciones in filtros_a.items()
}

# Mostrar resultados
for nombre, df in resultados_filtrados.items():
    print(f"\nConjunto {nombre} - Observaciones: {df.shape[0]}")
    display(df.head(1))


Conjunto A0 - Observaciones: 292781


,mrun,rango_edad,anio_ing_carr_ori,anio_ing_carr_act,tipo_inst_1,tipo_inst_2,tipo_inst_3,cod_inst,nomb_inst,nomb_carrera,region_sede,provincia_sede,comuna_sede,nivel_global,nivel_carrera_1,forma_ingreso,NOMBRE_REGION_INGRESO
3,98.0,15 a 19 años,2021,2021,Institutos Profesionales,Institutos Profesionales,Institutos Profesionales,111,IP DUOC UC,GASTRONOMIA,Metropolitana,SANTIAGO,SANTIAGO,Pregrado,Técnico de Nivel Superior,1- Ingreso Directo (regular),Metropolitana



Conjunto A - Observaciones: 127848


,mrun,rango_edad,anio_ing_carr_ori,anio_ing_carr_act,tipo_inst_1,tipo_inst_2,tipo_inst_3,cod_inst,nomb_inst,nomb_carrera,region_sede,provincia_sede,comuna_sede,nivel_global,nivel_carrera_1,forma_ingreso,NOMBRE_REGION_INGRESO
3,98.0,15 a 19 años,2021,2021,Institutos Profesionales,Institutos Profesionales,Institutos Profesionales,111,IP DUOC UC,GASTRONOMIA,Metropolitana,SANTIAGO,SANTIAGO,Pregrado,Técnico de Nivel Superior,1- Ingreso Directo (regular),Metropolitana



Conjunto A1 - Observaciones: 79104


,mrun,rango_edad,anio_ing_carr_ori,anio_ing_carr_act,tipo_inst_1,tipo_inst_2,tipo_inst_3,cod_inst,nomb_inst,nomb_carrera,region_sede,provincia_sede,comuna_sede,nivel_global,nivel_carrera_1,forma_ingreso,NOMBRE_REGION_INGRESO
43,984.0,15 a 19 años,2021,2021,Universidades,Universidades CRUCH,Universidades Estatales CRUCH,76,UNIVERSIDAD DE LA FRONTERA,ENFERMERIA,La Araucanía,CAUTIN,TEMUCO,Pregrado,Profesional Con Licenciatura,1- Ingreso Directo (regular),La Araucanía


In [7]:
resultados_filtrados['df

,mrun,rango_edad,anio_ing_carr_ori,anio_ing_carr_act,tipo_inst_1,tipo_inst_2,tipo_inst_3,cod_inst,nomb_inst,nomb_carrera,region_sede,provincia_sede,comuna_sede,nivel_global,nivel_carrera_1,forma_ingreso,NOMBRE_REGION_INGRESO
43,984.0,15 a 19 años,2021,2021,Universidades,Universidades CRUCH,Universidades Estatales CRUCH,76,UNIVERSIDAD DE LA FRONTERA,ENFERMERIA,La Araucanía,CAUTIN,TEMUCO,Pregrado,Profesional Con Licenciatura,1- Ingreso Directo (regular),La Araucanía
78,1703.0,15 a 19 años,2021,2021,Universidades,Universidades CRUCH,Universidades Estatales CRUCH,72,UNIVERSIDAD DE VALPARAISO,ARQUITECTURA,Valparaíso,VALPARAISO,VALPARAISO,Pregrado,Profesional Con Licenciatura,1- Ingreso Directo (regular),Valparaíso
82,1745.0,15 a 19 años,2021,2021,Universidades,Universidades CRUCH,Universidades Estatales CRUCH,83,UNIVERSIDAD DE PLAYA ANCHA DE CIENCIAS DE LA E...,DERECHO LICENCIATURA EN CIENCIAS JURIDICAS Y S...,Valparaíso,VALPARAISO,VALPARAISO,Pregrado,Profesional Con Licenciatura,1- Ingreso Directo (regular),Valparaíso
117,2430.0,15 a 19 años,2021,2021,Universidades,Universidades CRUCH,Universidades Estatales CRUCH,71,UNIVERSIDAD DE SANTIAGO DE CHILE,PERIODISMO / LICENCIATURA EN COMUNICACION SOCIAL,Metropolitana,SANTIAGO,ESTACION CENTRAL,Pregrado,Profesional Con Licenciatura,1- Ingreso Directo (regular),Metropolitana
170,3630.0,15 a 19 años,2021,2021,Universidades,Universidades CRUCH,Universidades Privadas CRUCH,3,UNIVERSIDAD DIEGO PORTALES,PSICOLOGIA,Metropolitana,SANTIAGO,SANTIAGO,Pregrado,Profesional Con Licenciatura,1- Ingreso Directo (regular),Metropolitana
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1291552,26379976.0,15 a 19 años,2021,2021,Universidades,Universidades Privadas,Universidades Privadas,38,UNIVERSIDAD ADVENTISTA DE CHILE,PEDAGOGIA EN EDUCACION GENERAL BASICA,Ñuble,DIGUILLIN,CHILLAN,Pregrado,Profesional Con Licenciatura,1- Ingreso Directo (regular),Ñuble
1293404,NaN,15 a 19 años,2021,2021,Universidades,Universidades Privadas,Universidades Privadas,9,UNIVERSIDAD DEL ALBA,NUTRICION Y DIETETICA,Coquimbo,ELQUI,LA SERENA,Pregrado,Profesional Con Licenciatura,1- Ingreso Directo (regular),Coquimbo
1293568,NaN,15 a 19 años,2021,2021,Universidades,Universidades Privadas,Universidades Privadas,9,UNIVERSIDAD DEL ALBA,TECNOLOGIA MEDICA,Metropolitana,SANTIAGO,PROVIDENCIA,Pregrado,Profesional Con Licenciatura,1- Ingreso Directo (regular),Metropolitana
1293953,NaN,15 a 19 años,2021,2021,Universidades,Universidades Privadas,Universidades Privadas,42,UNIVERSIDAD CATOLICA CARDENAL RAUL SILVA HENRI...,PSICOLOGIA,Metropolitana,SANTIAGO,SANTIAGO,Pregrado,Profesional Con Licenciatura,1- Ingreso Directo (regular),Metropolitana


In [26]:
# Lectura conjunto B

year = 2021
path_demre = Path(f"../data/raw/{year}")
list(path_demre.glob('*'))

[WindowsPath('../data/raw/2021/20230802_Matrícula_Ed_Superior_2021_PUBL_MRUN.csv'),
 WindowsPath('../data/raw/2021/A_INSCRITOS_PUNTAJES_PDT_2021_PUB_MRUN.csv'),
 WindowsPath('../data/raw/2021/establecimientos'),
 WindowsPath('../data/raw/2021/inmuebles_ies'),
 WindowsPath('../data/raw/2021/IVM_Establecimientos_2021.xlsx')]

In [27]:
# Definir el nombre del archivo
path_set_B = path_demre / 'A_INSCRITOS_PUNTAJES_PDT_2021_PUB_MRUN.csv'


year_B = int(re.search(r'_(\d{4})_', str(path_set_B)).group(1))

# Seleccionar columnas relevantes
columnas_B = [
    "MRUN", "RBD","CODIGO_REGION_EGRESO" ,"NOMBRE_REGION_EGRESO", "PTJE_RANKING","PTJE_NEM",
] + ["PROM_CM_ACTUAL" if year_B<2023 else "PROMEDIO_CM_MAX"]

# Leer el archivo
set_b = pd.read_csv(path_set_B ,
                   sep=';', encoding='utf-8',
                   dtype={'RBD': str, 'NOMBRE_REGION_EGRESO': str},
                   usecols=columnas_B
                   ).rename(columns={'MRUN': 'mrun','PROMEDIO_CM_MAX':'PROM_CM_ACTUAL'})

set_b['NOMBRE_REGION_EGRESO'] = set_b['NOMBRE_REGION_EGRESO'].replace(region_dict)
set_b['RBD'] = pd.to_numeric(set_b['RBD'], errors='coerce').dropna().astype(int)

# Mostrar las primeras filas
set_b.head(1)

,mrun,RBD,CODIGO_REGION_EGRESO,NOMBRE_REGION_EGRESO,PTJE_NEM,PTJE_RANKING,PROM_CM_ACTUAL
0,3703667,5654.0,9,La Araucanía,496,497,"674,5"


In [27]:
# Lectura conjunto C
path_set_C = path_establecimientos / 'layer_establecimientos_educacion_escolar_20220309024120.shp'
columnas_C = ["RBD", "NOM_RBD", "COD_REG_RB", "TIPO_DEPEN", "LATITUD", "LONGITUD"]

set_c = gpd.read_file(path_set_C)[columnas_C].rename(columns={'LATITUD': 'LATITUD_COL', 'LONGITUD': 'LONGITUD_COL'})
set_c['RBD'] = pd.to_numeric(set_c['RBD'], errors='coerce').dropna().astype(int)
set_c['COD_REG_RB'] = pd.to_numeric(set_c['COD_REG_RB'], errors='coerce')
set_c.head(1)

,RBD,NOM_RBD,COD_REG_RB,TIPO_DEPEN,LATITUD_COL,LONGITUD_COL
0,8484,ESCUELA PUERTO TORO,12.0,1.0,-55.083172,-67.076218


In [28]:
# Lectura conjunto D
path_set_D = path_inmuebles / 'layer_establecimientos_de_educacion_superior_20220309024111.shp'
columnas_D = ["NOMBRE_INS", "REGIÓN", "COMUNA", "LATITUD", "LONGITUD", "TIPO_INST"]

set_d = gpd.read_file(path_set_D)[columnas_D].rename(columns={'LATITUD': 'LATITUD_UNI', 'LONGITUD': 'LONGITUD_UNI'})
set_d['TIPO_INST'] = set_d['TIPO_INST'].str.capitalize()
set_d['REGIÓN'] = set_d['REGIÓN'].replace(region_dict)
set_d = set_d.drop_duplicates(subset=['NOMBRE_INS', 'REGIÓN', 'COMUNA'], keep='first')

set_d.head(1)

,NOMBRE_INS,REGIÓN,COMUNA,LATITUD_UNI,LONGITUD_UNI,TIPO_INST
0,CFT DE LA REGIÓN DE TARAPACÁ,Tarapacá,ALTO HOSPICIO,-20.283158,-70.094432,Centros de formación técnica estatales


In [29]:
# Lectura conjunto E
list(path_ivm.glob('*'))

[WindowsPath('../data/ivm/IVM_Establecimientos_2020.xlsx'),
 WindowsPath('../data/ivm/IVM_Establecimientos_2021.xlsx'),
 WindowsPath('../data/ivm/IVM_Establecimientos_2022.xlsx'),
 WindowsPath('../data/ivm/IVM_Establecimientos_2023.xlsx'),
 WindowsPath('../data/ivm/IVM_Establecimientos_2024.xlsx')]

In [30]:
# Definir el nombre del archivo
path_set_E = path_ivm / 'IVM_Establecimientos_2021.xlsx'
year_E = int(re.search(r'Establecimientos_(\d{4})', str(path_set_E)).group(1))

# Leer el archivo
set_e = pd.read_excel(path_set_E, sheet_name='Media')

# Procesar duplicados
set_e = (
    set_e.groupby("ID_RBD", group_keys=False)
    .agg({
        "N EVALUADO": "sum",
        "IVM Bajo": "sum",
        "IVM Medio": "sum",
        "IVM Alto": "sum",
        "IVM Muy Alto": "sum"
    })
    .assign(
        IVM_Establecimiento=lambda df: (
            (set_e.loc[set_e['ID_RBD'].isin(df.index), "IVM Establecimiento"] * 
             set_e.loc[set_e['ID_RBD'].isin(df.index), "N EVALUADO"]).groupby(set_e["ID_RBD"]).sum()
            / df["N EVALUADO"]
        )
    )
    .reset_index()
)

# Definir alta vulnerabilidad por año
filtros_e = {
    2020: 21.42601 ,
    2021: 20.03805,
    2022: 20.03805,
    2023: 18.33834,
    2024: 19.35458
}


set_e['valor_corte'] = filtros_e[year_E]


# Mostrar las primeras filas
set_e.head(1)

,ID_RBD,N EVALUADO,IVM Bajo,IVM Medio,IVM Alto,IVM Muy Alto,IVM_Establecimiento,valor_corte
0,1,403,118,30,41,214,23.798542,20.03805


In [31]:
# Guardar Datos
output_path = 'data/clean_data/'
os.makedirs(output_path, exist_ok=True)


resultados_filtrados['A1'].to_csv(output_path + 'set_a1.csv', index=False)
set_a.to_csv(output_path + 'set_a.csv', index=False)
set_b.to_csv(output_path + 'set_b.csv', index=False)
set_c.to_csv(output_path + 'set_c.csv', index=False)    
set_d.to_csv(output_path + 'set_d.csv', index=False)
set_e.to_csv(output_path + 'set_e.csv', index=False)

### Juntar Datos: Conjunto de datos ABCDE

In [32]:
# Función para rellenar las coordenadas de los colegios
def fill_university_coordinates(df, universities_db):
    """
    Fill missing university coordinates using hierarchical matching:
    1. Exact match (name + region + commune)
    2. Name + region match
    3. Name + commune match
    4. Region + commune match

    Nota: universities_db debe tener las columnas originales (LATITUD, LONGITUD)
    """
    # Verificar si hay valores NA para llenar
    na_rows = df[df['LATITUD_UNI'].isna()]
    if na_rows.empty:
        print("No hay valores NA en las coordenadas universitarias")
        return df

    print(f"\nFilling coordinates for {len(na_rows)} missing entries...")
    fill_count = 0

    for index, row in na_rows.iterrows():
        # Try exact match first
        match = universities_db[
            (universities_db['NOMBRE_INS'] == row['nomb_inst']) &
            (universities_db['REGIÓN'] == row['NOMBRE_REGION_INGRESO']) &
            (universities_db['COMUNA'] == row['comuna_sede'])
        ]

        # If no exact match, try less strict matches
        if match.empty:
            match = universities_db[
                (universities_db['NOMBRE_INS'] == row['nomb_inst']) &
                (universities_db['REGIÓN'] == row['NOMBRE_REGION_INGRESO'])
            ]

        if match.empty:
            match = universities_db[
                (universities_db['NOMBRE_INS'] == row['nomb_inst']) &
                (universities_db['COMUNA'] == row['comuna_sede'])
            ]

        if match.empty:
            match = universities_db[
                (universities_db['REGIÓN'] == row['NOMBRE_REGION_INGRESO']) &
                (universities_db['COMUNA'] == row['comuna_sede'])
            ]

        # If found any match, fill the coordinates (usamos las columnas originales)
        if not match.empty:
            df.at[index, 'LATITUD_UNI'] = match['LATITUD'].iloc[0]  # <-- Cambiado de LATITUD_UNI a LATITUD
            df.at[index, 'LONGITUD_UNI'] = match['LONGITUD'].iloc[0] # <-- Cambiado de LONGITUD_UNI a LONGITUD
            fill_count += 1

    print(f"Successfully filled {fill_count} missing coordinates")
    remaining_na = df['LATITUD_UNI'].isna().sum()
    print(f"Remaining NA values: {remaining_na}")

    return df

# Cálculo de distancias entre colegio y universidad
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    return 6371.0 * 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))

In [ ]:
# Junta de datos

# Conjunto AB
set_a1 = resultados_filtrados['A1']
set_ab = pd.merge(set_b,set_a1,on="mrun", how="inner").loc[lambda x: x['NOMBRE_REGION_EGRESO']!=' ']

# Conjunto ABC
set_abc = pd.merge(set_ab, set_c, on='RBD', how='inner')

# Conjunto ABCD
set_abcd = pd.merge(
    set_abc.rename(columns={'LATITUD': 'LATITUD_COL', 'LONGITUD': 'LONGITUD_COL'}),
    set_d.rename(columns={'LATITUD': 'LATITUD_UNI', 'LONGITUD': 'LONGITUD_UNI'}),
    left_on=['nomb_inst', 'NOMBRE_REGION_INGRESO', 'comuna_sede'],
    right_on=['NOMBRE_INS', 'REGIÓN', 'COMUNA'],
    how='left'
)

# Llenar coordenadas de universidades
set_abcd2 = fill_university_coordinates(set_abcd,  set_d.rename(columns={'LATITUD_UNI': 'LATITUD','LONGITUD_UNI': 'LONGITUD'}))

# Conjunto ABCDE
set_abcde = pd.merge(set_abcd2, set_e, left_on='RBD', right_on='ID_RBD', how='left')

# Manejo de valores nulos en IVM para colegios particulares pagados
valor_corte = set_e['valor_corte'][0]
set_abcde.loc[(set_abcde["IVM_Establecimiento"].isnull()) & (set_abcde["TIPO_DEPEN"] == 3), "IVM_Establecimiento"] = valor_corte - 1

# Distancia entre colegio y universidad
set_abcde['DISTANCIA'] = set_abcde.apply(
    lambda row: haversine(row['LATITUD_COL'], row['LONGITUD_COL'], row['LATITUD_UNI'], row['LONGITUD_UNI']),
    axis=1
)

# Obtener la cantidad de observaciones
cantidad_observaciones = set_abcde.shape[0]
print(f"Cantidad de observaciones: {cantidad_observaciones}")


# Guardar Datos
output_path = 'data/clean_data/'
os.makedirs(output_path, exist_ok=True)

set_ab.to_csv(output_path + 'set_ab.csv', index=False)
set_abc.to_csv(output_path + 'set_abc.csv', index=False)
set_abcd.to_csv(output_path + 'set_abcd.csv', index=False)
set_abcde.to_csv(output_path + 'set_abcde.csv', index=False)


Filling coordinates for 50976 missing entries...
Successfully filled 33634 missing coordinates
Remaining NA values: 17342
Cantidad de observaciones: 77366


In [34]:
# Guardar Datos
output_path = 'data/clean_data/'
os.makedirs(output_path, exist_ok=True)

set_ab.to_csv(output_path + 'set_ab.csv', index=False)
set_abc.to_csv(output_path + 'set_abc.csv', index=False)
set_abcd.to_csv(output_path + 'set_abcd.csv', index=False)
set_abcde.to_csv(output_path + 'set_abcde.csv', index=False)